In [1]:
import os
import sys
import glob
import zlib
import math
import pickle
import pymzml
import numpy as np
import pandas as pd
import _pickle as cpickle
import matplotlib.pyplot as plt
from importlib import reload
from collections import defaultdict as ddict
from scipy.ndimage.filters import gaussian_filter
from tensorly.decomposition import non_negative_parafac as nnp

sys.path.append(os.getcwd()+"../workflow/scripts/auxiliary/")
import LC_IM_MS_TensorAnalysis as hx

library_info=pd.read_csv("../resources/library_info/library_info.csv")

timepoints = [0, 10, 16, 26, 40, 65, 105, 160, 270, 430, 690, 1050, 1680, 2640, 4200, 6600, 11400, 18900, 33600, 62220, 100800]
tp_names = ["UN", "10s", "16s", "26s", "40s", "65s", "105s", "160s", "4m30", "7m10", "11m30s", "17m30s", "28m", "44m", "70m", "1hr50", "3hr10", "5hr15", "9hr20", "17hr45", "28hr"]

line = 99

tp_inputs = [sorted([fn for fn in glob.glob("../resources/tensors/"+str(line)+"_*") if tp_name in fn]) for tp_name in tp_names]

sample_un = hx.limit_read(tp_inputs[0][0])

ModuleNotFoundError: No module named 'LC_IM_MS_TensorAnalysis'

In [2]:
os.getcwd()

'/Users/rwl0960/Documents/hdx/HDX_LIMIT-Pipeline/notebooks'

In [2]:
mzml_gz = glob.glob("../isolate_tensors_redo/mzmls/*")[0]

In [7]:
msrun = pymzml.run.Reader(mzml_gz)

In [12]:
scan = msrun[1]

In [27]:
scan.peaks('reprofiled'), scan.peaks('centroided')

([(1743.74, 0.00016894625880781327),
  (1743.742, 0.0002948384815508044),
  (1743.744, 0.0005078168361451884),
  (1743.746, 0.0008632117982435794),
  (1743.748, 0.0014481547812677273),
  (1743.75, 0.0023977281391522275),
  (1743.752, 0.003918070447263067),
  (1743.754, 0.006318760336716645),
  (1743.756, 0.010057241276470189),
  (1743.758, 0.01579840334628141),
  (1743.76, 0.024492598801553654),
  (1743.762, 0.03747519336936492),
  (1743.764, 0.05659007102020442),
  (1743.766, 0.08433812434638953),
  (1743.768, 0.12404948914913891),
  (1743.77, 0.18007497566516414),
  (1743.772, 0.2579877542679078),
  (1743.774, 0.36478096014828754),
  (1743.776, 0.5090407790279414),
  (1743.778, 0.7010683001799476),
  (1743.78, 0.9529177759160864),
  (1743.782, 1.2783149535867757),
  (1743.784, 1.6924180667863111),
  (1743.786, 2.2113871369051576),
  (1743.788, 2.851735539569207),
  (1743.79, 3.629452020927116),
  (1743.792, 4.558901553841009),
  (1743.794, 5.651538768106768),
  (1743.796, 6.914496336

In [69]:
scan.measured_precision

5e-06

In [20]:
type(sample_un[2])

list

In [4]:
measured_precision = 0.000001
internal_precision = 1000

def reprofile(tensor): 
    out = []
    for scan in tensor[2]:
        tmp = ddict(int)
        for mz, i in scan:
            # Let the measured precision be 2 sigma of the signal width
            # When using normal distribution
            # FWHM = 2 sqt(2 * ln(2)) sigma = 2.3548 sigma
            s = mz * measured_precision * 2  # in before 2
            s2 = s * s
            #print(mz, i, s, s2)
            floor = mz - 5.0 * s  # Gauss curve +- 3 sigma
            ceil = mz + 5.0 * s
            #print(floor, ceil)
            ip = internal_precision / 4
            # more spacing, i.e. less points describing the gauss curve
            # -> faster adding
            for _ in range(int(round(floor * ip)), int(round(ceil * ip)) + 1):
                #print(_)
                if _ % int(5) == 0:
                    a = float(_) / float(ip)
                    y = i * math.exp(-1 * ((mz - a) * (mz - a)) / (2 * s2))
                    tmp[a] += y
        out.append(np.asarray([[key, tmp[key]] for key in list(tmp.keys())]))
    return out

In [5]:
reprofiled = reprofile(sample_un)

In [18]:
reprofiled[0][0, 1]

0.4251708220302574

In [2]:
#Generate rt-group ics

paths = glob.glob("data/new_tensors/"+str(line)+"_*")    
tp_inputs = [sorted([fn for fn in paths if tp_name in fn]) for tp_name in tp_names]
name = library_info.iloc[line]['name']

t = hx.TensorGenerator(name, library_info, tp_inputs, timepoints, n_factors_high=9)
t.generate_tensors()
hx.limit_write(t.all_tp_clusters, "data/new_ic_time_series/"+t.name+"_all_tp_clusters.cpickle.zlib")

Starting 0 seconds, 1 of 21
Sourcefile: data/new_tensors/99_180604_Mix2_MES_nonlin_UN.mzML.gz.cpickle.zlib, 1 of 3
File: data/new_tensors/99_180604_Mix2_MES_nonlin_UN.mzML.gz.cpickle.zlib
1 of 9 T+0.09103226661682129
2 of 9 T+0.23068499565124512
3 of 9 T+0.3187999725341797
4 of 9 T+0.38086390495300293
5 of 9 T+0.44152402877807617
6 of 9 T+0.4985496997833252
7 of 9 T+0.5636248588562012
8 of 9 T+0.6226117610931396
9 of 9 T+0.6837739944458008
Sourcefile: data/new_tensors/99_180604_Mix2_MES_nonlin_UN2.mzML.gz.cpickle.zlib, 2 of 3
File: data/new_tensors/99_180604_Mix2_MES_nonlin_UN2.mzML.gz.cpickle.zlib
1 of 9 T+0.08051204681396484
2 of 9 T+0.20624923706054688
3 of 9 T+0.3248860836029053
4 of 9 T+0.3814399242401123
5 of 9 T+0.4334580898284912
6 of 9 T+0.5027101039886475
7 of 9 T+0.5544948577880859
8 of 9 T+0.6230530738830566
9 of 9 T+0.6884551048278809
Sourcefile: data/new_tensors/99_180604_Mix2_MES_nonlin_UN3.mzML.gz.cpickle.zlib, 3 of 3
File: data/new_tensors/99_180604_Mix2_MES_nonlin_UN3

KeyboardInterrupt: 

In [ ]:
#PO and bokeh plot
    
name = library_info.iloc[line]['name']

all_tp_clusters = hx.limit_read("data/new_ic_time_series/"+name+"_all_tp_clusters.cpickle.zlib")

po = hx.PathOptimizer(name, all_tp_clusters, library_info, timepoints, n_undeut_runs = 3, old_data_dir = "/Users/rwl0960/Documents/hdx/local_LIMIT/data/old_data/")

po.optimize_paths()

source_frame, old_df = po.bokeh_plot("plots/"+name+"_new_time_series.html")